In [129]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [130]:
urls = []
for i in range(1,329632):
    urls.append('https://stackoverflow.com/questions?pagesize=50&page=' + str(i) + '&sort=newest')

In [131]:
def getWebText(link):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    result = requests.get(link, headers=headers)
    webpage = result.content.decode()
    return(webpage)

In [145]:
def getInfo(url):
    soup = BeautifulSoup(getWebText(url), 'html.parser').find('div', attrs={'id': 'questions'})
    qs = soup.find_all('div', attrs={'class':'question-summary'}) + soup.find_all('div', attrs={'class':'question-summary tagged-interesting'})
    df = pd.DataFrame()
    
    for q in qs:
        
        info      = {}

        info['q_id']      = q.get('id')
        info['q_votes']   = q.find('div').find('div').find('div').find('div').find('span').find('strong').text
        try:
            info['q_answers'] = q.find('div').find('div').find('div', attrs={'class':'status unanswered'}).find('strong').text
            info['q_answer_accepted'] = 0
        except AttributeError:
            try:
                info['q_answers'] = q.find('div').find('div').find('div', attrs={'class':'status answered'}).find('strong').text
                info['q_answer_accepted'] = 0
            except AttributeError:
                info['q_answers'] = q.find('div').find('div').find('div', attrs={'class':'status answered-accepted'}).find('strong').text
                info['q_answer_accepted'] = 1
        try:
            info['q_views']   = q.find('div').find('div', attrs={'class':'views '}).get('title')
        except:
            try:
                info['q_views']   = q.find('div').find('div', attrs={'class':'views warm'}).get('title')
            except:
                try:
                    info['q_views']   = q.find('div').find('div', attrs={'class':'views hot'}).get('title')
                except:
                    info['q_views']   = q.find('div').find('div', attrs={'class':'views supernova'}).get('title')
        info['q_views']   = info['q_views'].replace(' views','')
        info['q_url']     = q.find('div', attrs={'class':'summary'}).find('h3').find('a').get('href')
        info['q_title']   = q.find('div', attrs={'class':'summary'}).find('h3').find('a').text
        try:
            info['q_tags']    = ','.join(q.find('div', attrs={'class':'summary'}).find_all('div')[1].get('class')).replace('tags,','').replace('t-','')
        except IndexError:
            info['q_tags']    = ''
        try:
            info['q_time']    = q.find('div', attrs={'class':'started fr'}).find('div').find('div', attrs={'class':'user-action-time'}).find('span').get('title')
        except:
            info['q_time']    = ''
        try:
            info['q_user_url']      = q.find('div', attrs={'class':'started fr'}).find('div').find('div', attrs={'class':'user-gravatar32'}).find('a').get('href')
        except AttributeError:
            df = pd.concat([df, pd.DataFrame(info, index = [0])])
            continue
        info['q_user_repscore'] = q.find('div', attrs={'class':'started fr'}).find('div', attrs={'class':'user-details'}).find('span', attrs={'class':'reputation-score'}).text

        titles = []
        for sp in q.find('div', attrs={'class':'started fr'}).find('div', attrs={'class':'user-details'}).find_all('span')[1:]:
            titles.append(sp.get('title'))
        titles = [x for x in titles if x is not None]
        badges = {'bronze':0, 'silver':0, 'gold':0}
        for b in titles:
            bnum  = b.split()[0]
            bname = b.split()[1]
            badges[bname] = bnum

        info['q_user_badges_bronze']   = badges['bronze']
        info['q_user_badges_silver']   = badges['silver']
        info['q_user_badges_gold']     = badges['gold']
        df = pd.concat([df, pd.DataFrame(info, index = [0])])

    return(df)

In [234]:
questions = pd.DataFrame()
p1 = 0
p2 = 0      
for u in urls:
    #time.sleep(0.5)
    p2 = p2+1
    print(u, end = '\r')#print urls
    frame = getInfo(u)
    questions = pd.concat([questions, frame]) 
    if p2 >= (p1+1000): #export data every 1000 rows to be safe
        questions.to_csv('~/data_' + str(p1) + '_' + str(p2) + '.csv')
        questions = pd.DataFrame()
        p1 = p2
        